In [1]:
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt

import sys
sys.path.append(".")
import os
# os.environ["OPENAI_API_KEY"]="sk-xxxxxx"

from agent.agents import agent_factory

In [2]:
def show_image(image):
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [7]:
import requests

img_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg"
image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")


prompt="""task: I want to get in the car."""


In [8]:
agent = agent_factory("SegVLM")

In [9]:
plan_code, masks, info_dict = agent.plan(prompt, image, return_info=True)

In [ ]:
plan_code

"To get into the car, the robot needs to open the car door first. Based on the image you provided and following the given instructions, I'll write a plan that instructs the robot to perform the necessary operation to open the car door.\n\nLet's assume the car door is represented by a highlighted region. If any of the highlighted regions by the numbers correspond to the car door handle, that is the region we will use for the `open` operation.\n\n```python\n# We'll use open operation on the door handle region.\n# In case the region corresponding to the door handle is not highlighted,\n# this plan would need adjustment with the correct region number.\nopen(regions[handle_region_number])\n```\n\nSince I can't identify the door handle by a specific number based on the instructions, you must replace `handle_region_number` with the correct region number representing the car door handle before executing the plan."